In [1]:
%cd /home/david/Desktop/antenna_arrays

/home/david/Desktop/antenna_arrays


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from true_function import *
import op_true_func as op
import torch_function as t_op

In [3]:
Yant0, Zant0 = pd.read_csv('check_shapes/shape 0/Given Yant0.csv', header= None).values.flatten(), pd.read_csv('check_shapes/shape 0/Given Zant0.csv', header = None).values.flatten()

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
ParamsArray = t_op.InitializeParamsArray(device = device)

In [6]:
# Put Yant0 Zant0 to gpu
yant0, zant0 = torch.tensor(Yant0).to(device), torch.tensor(Zant0).to(device)

In [7]:

cost = t_op.CostFunArray(yant0, zant0, ParamsArray, device)

In [8]:
cost.backward()

In [9]:
print(yant0.grad, zant0.grad)

None None


In [9]:
ParamsArray = InitializeParamsArray()

In [10]:
CostFunArray(Yant0, Zant0, ParamsArray)

-12131.25785906924

In [3]:
import op_true_func as op

In [5]:
from torch.utils.data import DataLoader
from datasets.AntDataset import *

In [7]:
train_loader = DataLoader(AntDataset('data/large/YZ_Large_70.npz'))

In [8]:
def reformat_ant_format(ant):
    """
    Reformats the 2048 to 2 distinct arrays, trimming the 0
    """
    Yant, Zant = ant[:1024], ant[1024:]
    trim_yant, trim_zant = np.trim_zeros(Yant, 'b'), np.trim_zeros(Zant, 'b')
    return trim_yant, trim_zant

In [10]:
sample = next(iter(train_loader))
sample

[tensor([[-30.9829, -30.3919, -29.8009,  ...,   0.0000,   0.0000,   0.0000]]),
 tensor([-12131.3057])]

In [13]:
ParamsArray = InitializeParamsArray()

In [26]:
inp, cost = sample
inp = inp.squeeze(0).numpy()
trimmed = reformat_ant_format(inp)
print(CostFunArray(inp[:1024], inp[1024:], ParamsArray))
print(CostFunArray(trimmed[0], trimmed[1], ParamsArray))

-4.368310549902555
-12131.259246393902


In [39]:
cost.item()

-12131.3056640625

In [42]:
import time
results = np.zeros(shape = (1000, 5)) # true cost (from the data), true_cost func, op function cost, true cost time, op func time, 1000 examples
device = 'cuda' if torch.cuda.is_available() else 'cpu'
ParamsArray = InitializeParamsArray()
OP_ParamsArray = t_op.InitializeParamsArray(device=device)

In [43]:
for i, (inp, cost) in enumerate(train_loader):
    if i == 1000: break
    
    print(f'{i + 1} Iteration')
    results[i, 0] = cost.item()  # true cost (from the data)
    cloned = inp.clone()
    trim_yant, trim_zant = reformat_ant_format(inp.squeeze(0).numpy())
    
    # Time the execution of CostFunArray
    start_time = time.time()
    results[i, 1] = CostFunArray(trim_yant, trim_zant, ParamsArray)
    end_time = time.time()
    results[i, 3] = end_time - start_time  # true cost time
    
    op_yant, op_zant = torch.from_numpy(trim_yant.copy()).to(device), torch.from_numpy(trim_zant.copy()).to(device)
    
    # Time the execution of t_op.CostFunArray
    start_time = time.time()
    results[i, 2] = t_op.CostFunArray(op_yant, op_zant, OP_ParamsArray, device)
    end_time = time.time()
    results[i, 4] = end_time - start_time  # op func time

1 Iteration
2 Iteration
3 Iteration
4 Iteration
5 Iteration
6 Iteration
7 Iteration
8 Iteration
9 Iteration
10 Iteration
11 Iteration
12 Iteration
13 Iteration
14 Iteration
15 Iteration
16 Iteration
17 Iteration
18 Iteration
19 Iteration
20 Iteration
21 Iteration
22 Iteration
23 Iteration
24 Iteration
25 Iteration
26 Iteration
27 Iteration
28 Iteration
29 Iteration
30 Iteration
31 Iteration
32 Iteration
33 Iteration
34 Iteration
35 Iteration
36 Iteration
37 Iteration
38 Iteration
39 Iteration
40 Iteration
41 Iteration
42 Iteration
43 Iteration
44 Iteration
45 Iteration
46 Iteration
47 Iteration
48 Iteration
49 Iteration
50 Iteration
51 Iteration
52 Iteration
53 Iteration
54 Iteration
55 Iteration
56 Iteration
57 Iteration
58 Iteration
59 Iteration
60 Iteration
61 Iteration
62 Iteration
63 Iteration
64 Iteration
65 Iteration
66 Iteration
67 Iteration
68 Iteration
69 Iteration
70 Iteration
71 Iteration
72 Iteration
73 Iteration
74 Iteration
75 Iteration
76 Iteration
77 Iteration
78 Itera

In [45]:
result_df = pd.DataFrame(results, columns=['True Cost (from Data)', 'True Cost (from fun)', 'Op True Cost (from Torch)', 'True Cost Time', 'Torch True Cost Time'])
result_df

,True Cost (from Data),True Cost (from fun),Op True Cost (from Torch),True Cost Time,Torch True Cost Time
0,-12131.305664,-12131.259246,-12131.267578,6.646008,0.028911
1,-753.547546,-753.546092,-753.547424,6.703403,0.030354
2,-30103.195312,-30103.135886,-30103.134766,6.525104,0.029823
3,-5349.999512,-5349.972089,-5349.973145,6.375337,0.029623
4,-5900.174805,-5900.169258,-5900.175293,6.441304,0.029924
...,...,...,...,...,...
995,-4333.630371,-4333.614120,-4333.617676,6.173140,0.027165
996,-29530.310547,-29530.279616,-29530.275391,6.119851,0.027621
997,-273.087128,-273.086487,-273.087036,6.407505,0.028393
998,-597.462708,-597.459965,-597.460693,6.370972,0.028132


In [46]:
# Find out average difference
diff = results[:, 0] - results[:, 1]
np.mean(diff)

-0.03436577556283134

In [47]:
og_time = results[:, 3].mean()
op_time = results[:, -1].mean()

In [48]:
og_time

6.278073018074036

In [49]:
op_time

0.028532171726226806

In [52]:
diff = np.sqrt((results[:, 1] - results[:, 2])**2)
np.mean(diff)

0.002944670376793841